Loading Ollama model

In [1]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:latest")

Establishing connection to the Database

In [2]:
import yaml
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase

with open('../config/config.yml', 'r') as file:
    config = yaml.safe_load(file)

mysql_config = config.get('mysql')

username = mysql_config.get('username')
password = mysql_config.get('password')
host = mysql_config.get('host')
port = mysql_config.get('port')
database = mysql_config.get('database')

connection_string = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

db = SQLDatabase(engine)

Checking the connection with a database

In [3]:
db.run("SELECT * FROM employee")

"[(1, 'John', 'Doe', 'john.doe@example.com', Decimal('55000.00')), (2, 'Jane', 'Smith', 'jane.smith@example.com', Decimal('62000.00')), (3, 'Mike', 'Johnson', 'mike.johnson@example.com', Decimal('48000.00')), (4, 'Emily', 'Davis', 'emily.davis@example.com', Decimal('51000.00'))]"

In [15]:
from langchain.chains import create_sql_query_chain

sql_chain = create_sql_query_chain(llm, db)
question = "how many employees are there? You MUST RETURN ONLY MYSQL QUERIES."
response = sql_chain.invoke({"question": question})

Telling model how to extract the query from the previous response (need to parse the string)

In [16]:
from langchain_core.prompts import (SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate)
from langchain_core.output_parsers import StrOutputParser

system = SystemMessagePromptTemplate.from_template("""You are helpful AI assistant who answer user question based on the provided context.""")

prompt = """Answer user question based on the provided context ONLY! If you do not know the answer, just say "I don't know".
            ### Context:
            {context}

            ### Question:
            {question}

            ### Answer:"""

prompt = HumanMessagePromptTemplate.from_template(prompt)

messages = [system, prompt]
template = ChatPromptTemplate(messages)

qna_chain = template | llm | StrOutputParser()

def ask_llm(context, question):
    return qna_chain.invoke({'context': context, 'question': question})

In [17]:
from langchain_core.runnables import chain

@chain
def get_correct_sql_query(input):
    context = input['context']
    question = input['question']

    intruction = """
        Use above context to fetch the correct SQL query for following question
        {}

        Do not enclose query in ```sql and do not write preamble and explanation.
        You MUST return only single SQL query.
    """.format(question)

    response = ask_llm(context=context, question=intruction)

    return response

In [18]:
response = get_correct_sql_query.invoke({'context': response, 'question': question})
db.run(response)

'[(4,)]'

Building the chain that is capable of extracting the right query

In [19]:
from langchain_community.tools import QuerySQLDatabaseTool
from langchain_core.runnables import RunnablePassthrough 

execute_query = QuerySQLDatabaseTool(db=db)
sql_query = create_sql_query_chain(llm, db)

final_chain = (
    {'context': sql_query, 'question': RunnablePassthrough()}
    | get_correct_sql_query
    | execute_query
)

In [20]:
question = "how many employees are there? You MUST RETURN ONLY MYSQL QUERIES."

response = final_chain.invoke({'question': question})
print(response)

[(4,)]


Building an agent